In [1]:
from dataclasses import dataclass
import itertools
import logging
import random
import math
import numpy as np
import json
import pickle
import time
import torch
import sys
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
from probe_utils import *
from plot_utils import *
from tqdm import tqdm

from omegaconf import OmegaConf
from pathlib import Path
from torch import nn, Tensor
from torch.nn import functional as F
from typing import List, Optional, Tuple
import os
# os.chdir("/data/tianyu_guo/birth")
from data import DataArgs, Dataset, iterate_batches, make_dataset
from ihead_full_model import *

%load_ext autoreload
%autoreload 2


In [51]:
torch.cuda.set_device(0)
device = 'cuda:0'
run_path_server = "/data/tianyu/birth/gens/pre-iclr/dynamics/dormant_copy_long_train"
# run_path_server2="/data/tianyu_guo/birth/gens/special/dormant_copy_2"
model, cfg, x, y, ds, optim = load_model(run_path_local="/Users/guotianyu/GitHub/birth/gens/special/markov", run_path_server=run_path_server, bos_num=1, train_steps=10000, delim=0, n_layers=3, n_heads=1, no_attn_norm=(), no_ffn_norm=(), no_attn=(), no_ffn=(), linear_ffn=(), with_data=True, with_optim=True, data_path_local="/Users/guotianyu/GitHub/birth/data", data_path_server="/data/tianyu/birth/data", device=device)
hook = forward_hook([], '')

grads, params, updates = {}, {}, {}
optim.zero_grad()
pred, outputs_list = model.modified_forward_with_hook(x, hook)

grad_weight= torch.autograd.grad(outputs=outputs_list[0]['output'][0, 0, :].norm() ** 2, inputs=model.parameters(), create_graph=True, allow_unused=True)
norm_grads = {}
for (name, param), grad in zip(model.named_parameters(), grad_weight):
    if grad is not None:
        norm_grads[name] = grad

loss = F.cross_entropy(pred.flatten(0, 1), y.flatten(0, 1))
loss.backward()
for name, param in model.named_parameters():
    grads[name] = param.grad.detach().clone()
    params[name] = param.detach().clone()
optim.step()
for name, param in model.named_parameters():
    updates[name] = param.detach().clone() - params[name]
probs = get_oracle_predicts(x, ds)
triggers_pos = ds.get_triggers_pos(x.to('cpu'))
risk = get_risk(probs, pred, predict_in_logits=True, triggers_pos=triggers_pos)


In [43]:
norm_contributes = {}
for name, grad in norm_grads.items():
    norm_contributes[name] = (grad * updates[name]).sum().cpu().detach().item()

In [50]:
grads_copy = grads.copy()